In [43]:
# Load modules
import os
import pandas as pd
import re
import ast
import glob
import csv
import shutil
import numpy as np
import geopy
import pygmt
import pyproj
from geopy.distance import geodesic
from geopy.distance import distance
from os import system as sys

In [44]:
#psudo initialization file, this is the only thing we want users to be changing things in

#explicit path to your data: .ts
path = "C:/Users/19049/coding_files/data/CFM5.3_traces.ts"

#path to subset of your data, for your testing of this code: .ts
subset = "C:/Users/19049/coding_files/data/subset.ts"

#open path or subset
to_open = path

#folder you want the individual fault csv's to output in
output_individual_fault_csv = False
folder = "C:/Users/19049/coding_files/faults/"

#meta information for worldbuilder: 'strings'
model      = 'fault' 
wb_version = '0.5'

#these will eventually be read in (maybe), but for now they are specified by user: 'strings'
dip_point_list                = '[0,0]'
segment_length                = '29000'
segment_thickness             = '2000'
segment_angle                 = '89.9'
temp_model_type               = 'uniform'
uniform_model_temp            = '273'
composition_model_type        = 'smooth'
compositions_list             = '[0,1]'
side_distance_to_fault_center = '1000.0'
center_fractions_list         = '[0.1,0.1]'
side_fractions_list           = '[0.0,0.0]'

#orienting to your study area: integers
###Need to ground this in a coordinate system or something, since I have no idea where the 0 is for these measurements###
easting_adjust  = -345000
northing_adjust = -3870000

# format for ts file from SCEC community fault model: '*_fault',\n
faults_to_include = [
    'Panamint_Valley_fault',
    'Southern_Death_Valley_fault',
    'Black_Mountain_fault',
    'Owl_Lake_fault',
    'Hunter_Mountain_fault',
    'Ash_Hill_fault',
    'Independence_Sierra_Nevada_fault', #Gotta add the _fault identification bit cause of
    'Independence_fault',               #these two lines
    'Little_Lake_fault',
    'Airport_Lake_fault',
    'Eastern_Little_Lake_north_north_east_cross_fault',
    'Owens_Valley_fault',
    'Northern_Death_Valley_fault',
    #'-Casmalia_fault', #cause of this guy we gotta have the '-' at the beginning, not sure if this will bug the wb file, but i think it can be stripped later
    'thing'                             #if thing shows up in files, stuff is wack
    ]

In [46]:
#once features get cemented they will be added here for readability of main code



#makes an empty pandas dataframe to fill the adjusted values with
#for ts file from SCEC community fault model
#",'Elevation(m)'"
def make_empty_fault_df():
    
    global fault_dataframe
    fault_dataframe = pd.DataFrame([],columns = ['Easting(m)','Northing(m)'])

    
#the line being read is delimited and turned from a line of strings into a line of floats
#then it is apended to the fault_dataframe dataframe with the elements adjusted
#add ", float_list[2]" to end of fault_dataframe to log the elevation
#fault_dataframe.loc[len(fault_dataframe)]
def adjust_to_df():
    
        string_line = line.split()
        float_list = list(map(float, string_line))
        fault_dataframe.loc[len(fault_dataframe)] = float_list[0]+easting_adjust, float_list[1]+northing_adjust
        


def adjust_to_wb():
        
        string_line = line.split()
        float_list = list(map(float, string_line))
        coors = [float_list[0]+easting_adjust, float_list[1]+northing_adjust]
        coordinates.append(coors)  
        return coordinates
    
        

#takes the finished fault_dataframe df and names it the current fault trace, making it a csv    
def save_to_file():
    
    fault_dataframe.to_csv(folder+use_name+".csv")
    

    
def initialize(version_i):
    
    f.write('{')
    f.write('\n')
    f.write('\t')
    f.write('"version":"')
    f.write(version_i)
    f.write('"')
    #'"version":"0.5"'
    f.write(', \n')
    f.write('\t')
    f.write('"features":')
    f.write('\n')
    f.write('\t')
    f.write('[')
    f.write('\n')
    
    
    
def body_write_format(model_i, 
                      codename_i, 
                      dip_point_list_i, 
                      coordinates_i, 
                      segment_length_i, 
                      segment_thickness_i, 
                      segment_angle_i,
                      temp_model_type_i,
                      uniform_model_temp_i,
                      composition_model_type_i,
                      compositions_list_i, 
                      side_distance_to_fault_center_i, 
                      center_fractions_list_i, 
                      side_fractions_list_i
                      ):
     
    f.write('\t')
    f.write('{')
    f.write('"model":"')
    f.write(model_i)
    f.write('"')
    #'"model":"fault"'
    f.write(',')
    f.write(codename_i)
    f.write(',')
    f.write('"dip point":')
    f.write(dip_point_list_i)
    #'"dip point":[0,0]'
    f.write(', \n')
    f.write('\t')
    f.write('\t')
    f.write('"coordinates":[')
    f.write('\n')
    f.write(', \n'.join(str(i) for i in coordinates_i))
    f.write('\n')
    f.write('],')
    f.write('\n') #start of segment buisness
    f.write('\t')
    f.write('\t')
    f.write('"segments":')
    f.write('\n')
    f.write('\t')
    f.write('\t')
    f.write('[')
    f.write('\n')
    f.write('\t')
    f.write('\t')
    f.write('\t')
    f.write('{"length":')
    f.write(segment_length_i)
    f.write(',"thickness":[')
    f.write(segment_thickness_i)
    f.write('],"angle":[')
    f.write(segment_angle_i)
    f.write(']}')
    #f.write('{"length":29000,"thickness":[2000],"angle":[89.9]}')
    f.write('\n')
    f.write('\t')
    f.write('\t')
    f.write('],')
    f.write('\n')
    f.write('\t')
    f.write('\t')
    f.write('"temperature models":[{"model":"')
    f.write(temp_model_type_i)
    f.write('","temperature":')
    f.write(uniform_model_temp_i)
    f.write('}],')
    #f.write('"temperature models":[{"model":"uniform","temperature":273}],')
    f.write('\n')
    f.write('\t')
    f.write('\t')
    f.write('"composition models":')
    f.write('\n')
    f.write('\t')
    f.write('\t')
    f.write('[')
    f.write('\n')
    f.write('\t')
    f.write('\t')
    f.write('\t')
    f.write('{"model":"')
    f.write(composition_model_type_i)
    f.write('","compositions":')
    f.write(compositions_list_i)
    f.write(', "side distance fault center": ')
    f.write(side_distance_to_fault_center_i)
    f.write(', "center fractions":')
    f.write(center_fractions_list_i)
    f.write(', "side fractions":')
    f.write(side_fractions_list_i)
    f.write('}')
    #f.write('{"model":"smooth","compositions":[0,1], "side distance fault center": 1000.0, "center fractions":[0.1,0.1], "side fractions":[0.0,0.0]}')
    f.write('\n')
    f.write('\t')
    f.write('\t')
    f.write(']') #end of segment buisness
    f.write('\n')
    f.write('\t')
    f.write('\t')
    f.write('},') #has , that gets deleted when we cap the wb file
    f.write('\n')


    
def close_wb_file():
    
    f.write('\n')
    f.write('\t')
    f.write(']')
    f.write('\n')
    f.write('}')

In [42]:
#Takes the raw data from a ts SCEC community fault model file and writes it to a wb file and optionally individual csvs

#clears working file so we don't stack
for file in glob.glob("C:/Users/19049/coding_files/*"):
    if "wb_wip.txt" in file:
        os.remove(file)
        

#initial condidtions and intitiations of flags
Associated_with_current_fault = True
wb=[]
coordinates = []
included_fault_tracker = []
global name
print('Faults being transcribed: ')


#determine last line
with open(to_open) as f:
    for line in f:
        pass
    global last_line
    last_line = line

#initialize the wb file
with open("wb_wip.txt", "a") as f:
    initialize(wb_version)

    
with open(to_open) as f:
    for line in f:
        #purpose of this is that the last set of coordinates were not being outputted, 
        #since the output was dependent on a new fault being read in, so the above for
        #loop finds the last line and this if statement outputs the stuff if it is the 
        #last line. Isn't invoked in the full data read, but needed for robustness.
        if line == last_line:
            
            #here to differentiate between being done with the current set 
            #of coordinates for a given fault or not, setting it to false means we're 
            #done with that set of coordinates and can send the wrapped set to the wb file
            #by wrapped I mean the coordinates for a fault are all together within a 
            #list and can be associated with the name and whatever else of the fault
            Associated_with_current_fault = False
            is_number = False
            while Associated_with_current_fault == False:
                #sorts faults that we want from ones we don't based on the fault tracker flag
                if 'yes' in included_fault_tracker:
                    with open("wb_wip.txt", "a") as f:
                        codename = '"name":'+name
                        body_write_format(model, 
                                          codename, 
                                          dip_point_list, 
                                          coordinates, 
                                          segment_length, 
                                          segment_thickness, 
                                          segment_angle,
                                          temp_model_type,
                                          uniform_model_temp, 
                                          composition_model_type, 
                                          compositions_list, 
                                          side_distance_to_fault_center, 
                                          center_fractions_list, 
                                          side_fractions_list)
                        
                break
        #the beginning of each new fault will have this as the starting characters of 
        #that line and every coordinate until the start of the next fault will be associated with it, 
        if line.startswith("> "):
            Associated_with_current_fault = False
            is_number = False
            #this while loop breaks because even though we are evaluating a fault name, we don't know
            #if we want to look at the coordinates yet based on the included_fault_tracker list
            #(nothing in the list breaks the loop cause no 'yes'). Once we do want the coordinates, this  will change once we go
            #through every coordinate under a fault we want and hit a new fault name indicated by "> "
            #this will spit out the previous fault's coordinates and associates it with the name of the fault
            #
            while Associated_with_current_fault == False:
                if 'yes' in included_fault_tracker:
                    with open("wb_wip.txt", "a") as f:
                        codename = '"name":'+name
                        body_write_format(model, 
                                          codename, 
                                          dip_point_list, 
                                          coordinates, 
                                          segment_length, 
                                          segment_thickness, 
                                          segment_angle,
                                          temp_model_type,
                                          uniform_model_temp,
                                          composition_model_type, 
                                          compositions_list, 
                                          side_distance_to_fault_center, 
                                          center_fractions_list, 
                                          side_fractions_list)
                break
            included_fault_tracker = []
            for real_name in faults_to_include:                            
                if real_name in line:           
                    #for changing the fault name that is associated with each set of coordinates
                    name = real_name
                    #initiates and resets the coordinates variable in conjuntion with the name above
                    coordinates = []
                    print(name)
                    #if we have the name of one of the faults (from the faults_to_include list) 
                    #we want in the real_name line then we add a yes to the tracker
                    included_fault_tracker.append('yes') 
                else:
                    
                    included_fault_tracker.append('no')            
        elif line.startswith("#"): 
            #this file has 15 '#'s at the beginning and I wanted to be explicit with what happens there
            pass  
        
        else:     
            #
            is_number = True 
            while is_number == True:                        
                if 'yes' in included_fault_tracker: 
                    #changes the raw coordinate data to a comma seperated, adjusted to study area,
                    #list of floats in a list. Goes line by line and appends each 
                    #pair: format [[165833.0625, 173051.0], [165133.0, 172049.5],... ]
                    adjust_to_wb()
                else:                           
                    pass
                
                break
        #not done with the coordinates of a particular fault yet, so the while loop keeps going and once 
        #we hit a new fault name we will set the Associated_with_current_fault parameter to false
        #and that is the queue to take the coordinates and write them into the wb file
        Associated_with_current_fault = True  

        
#instead of differentiating which fault is the last one, I just cut off the last comma
with open("wb_wip.txt", "rb+") as f:
    f.seek(-3, os.SEEK_END)
    f.truncate()

    
#after we do all that we cap the wb file
with open("wb_wip.txt", "a") as f:
    close_wb_file()

    
#makes files containing each fault data
if output_individual_fault_csv == True:
    print('\n')
    print('Individual fault files going in: ' + folder)
    with open(to_open, encoding='utf-8') as f:
        for line in f:                                          
            if line.startswith("> "):                           
                is_number = False                               
                included_fault_tracker = [] 

                for real_name in faults_to_include:                            
                    if real_name in line:                       
                        print(real_name)
                        global use_name
                        use_name = real_name                                        
                        make_empty_fault_df()                   
                        included_fault_tracker.append('yes')     
                    else:                                       
                        included_fault_tracker.append('no')     
            elif line.startswith("#"):                          
                pass                                            
            else:                                               
                is_number = True                                
                while is_number == True:                        
                    if 'yes' in included_fault_tracker:         
                        adjust_to_df()                      

                    else:                           
                        pass
                    break
                save_to_file()   

Faults being transcribed: 
Black_Mountain_fault
Northern_Death_Valley_fault
Southern_Death_Valley_fault
Owl_Lake_fault
Ash_Hill_fault
Airport_Lake_fault
Hunter_Mountain_fault
Little_Lake_fault
Eastern_Little_Lake_north_north_east_cross_fault
Owens_Valley_fault
Panamint_Valley_fault
Independence_fault
Independence_Sierra_Nevada_fault


Individual fault files going in: C:/Users/19049/coding_files/faults/
Black_Mountain_fault
Northern_Death_Valley_fault
Southern_Death_Valley_fault
Owl_Lake_fault
Ash_Hill_fault
Airport_Lake_fault
Hunter_Mountain_fault
Little_Lake_fault
Eastern_Little_Lake_north_north_east_cross_fault
Owens_Valley_fault
Panamint_Valley_fault
Independence_fault
Independence_Sierra_Nevada_fault


In [5]:
#tool to delete working file
for file in glob.glob("C:/Users/19049/coding_files/*"):
    if "wb_wip.txt" in file:
        os.remove(file)

In [25]:
print(last_line)

156687.5625   3877500.7500   -76.058300000
